In [1]:
import pandas as pd
import numpy as np
import copy
import csv
import itertools

from tqdm import tqdm
from Bio import SeqIO, SearchIO 

In [2]:
rodeo = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/main_co_occur_rodeo_psi.csv', sep = ",")
rodeo = rodeo.loc[:,['Query','Protein_acc']]
rodeo

,Query,Protein_acc
0,WP_145853129.1,WP_187365872.1
1,WP_145853129.1,WP_187365873.1
2,WP_145853129.1,WP_145853125.1
3,WP_145853129.1,WP_145853126.1
4,WP_145853129.1,WP_145853127.1
...,...,...
11152,WP_015759610.1,WP_015759614.1
11153,WP_015759610.1,WP_015759615.1
11154,WP_015759610.1,WP_015759616.1
11155,WP_015759610.1,WP_015759617.1


__Посчитать, сколько раз во всех наших геномных регионах/оперонах встретилась доменная архитектура  "домен X - домен Y - ... - домен N"__

In [36]:
file = '/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/parsed_hmm_per_dom_tbl.txt'
res_domains = dict()
x = 10
count = 0
for res in tqdm(SearchIO.parse(file, 'hmmscan3-domtab')):
        res_domains[res.id] = {}
        for hit in res.hits:
            if hit.id[:4] == 'TIGR':
                    call = hit.description.split(":")[0]
            else:
                    call = hit.id
            for hsp in hit.hsps:
                overlap = False
                if res_domains[res.id].keys(): #not empty
                    names = copy.deepcopy(list(res_domains[res.id].keys()))
                    for name in names: #проверка на перекрытие с раннее добавленными генами 
                        for q_range in res_domains[res.id][name]:
                            if min(hsp.env_end, q_range[1]) - max(hsp.env_start, q_range[0]) > x:
                                overlap = True
                                break
                        if overlap:
                            break
                    if not overlap: 
                        if call not in res_domains[res.id]:
                            res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                            
                        if (hsp.env_start, hsp.env_end) not in res_domains[res.id][call]:
                                        #для случая мультидоменного белка с одинаковыми доменами: 
                                        #один и тот же белок выравнивается на несколько мест
                            res_domains[res.id][call].append((hsp.env_start, hsp.env_end))
                       
                else:
                    res_domains[res.id][call] = [(hsp.env_start, hsp.env_end)]
                    

10493it [00:23, 443.55it/s]


In [60]:
domains_names = {}
for key, value in res_domains.items():
    domains_names[key] = list(value.keys())

domains_names = pd.DataFrame({'domains': domains_names.values()}, index =res_domains.keys())
domains_names

,domains
WP_011566059.1,[MnhB]
WP_011566060.1,[2a6301s01]
WP_011566061.1,"[mycofact_glyco, Methyltransf_23, Methyltransf..."
WP_011566062.1,[Creatininase]
WP_011566063.1,[actino_HemFlav]
...,...
WP_160900598.1,"[DoxX_3, DoxX]"
WP_160901386.1,[DNA_alkylation]
WP_160900597.1,"[Pyr_redox_2, Reductase_C]"
WP_160900596.1,"[FAM70, DUF3040]"


In [61]:
domain_for_region = rodeo.Protein_acc.value_counts()
domain_for_region = pd.DataFrame(domain_for_region)
domain_for_region.columns = ['counts']
domain_for_region

,counts
WP_003873519.1,7
WP_003408976.1,5
WP_132992091.1,4
WP_009937382.1,4
WP_007773607.1,3
...,...
WP_194782143.1,1
WP_194782144.1,1
WP_228529047.1,1
WP_228529048.1,1


In [62]:
domain_for_region = domain_for_region.join(pd.DataFrame(domains_names)).dropna(subset = ['domains'])
domain_for_region['domains'] = [str(key)[1:-1] for key in domain_for_region['domains']]
domain_for_region = domain_for_region.groupby('domains').sum()
domain_for_region = domain_for_region.sort_values(by = ['counts'], ascending=False)
domain_for_region

,counts
domains,
'mycofactocin',665
'Mycofactocin_RRE',643
'mycofact_rSAM',615
'actino_HemFlav',491
'mycofact_glyco',458
...,...
"'LysR_substrate', 'HTH_1', 'HTH_24'",1
"'BPD_transp_1', 'permease_CysT'",1
"'BPD_transp_1', 'phosphate_pstC'",1


In [58]:
#domain_for_region.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_arch_regions.csv')

__Для оперонов:__

In [63]:
operon_prot_id = pd.read_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/operon_100_protIDs_rodeo.csv', sep = ",", index_col = 0)
operon_prot_id

,Query,Protein_acc
1,WP_145853129.1,WP_246115648.1
2,WP_145853129.1,WP_246115687.1
3,WP_145853129.1,WP_145853128.1
4,WP_145853129.1,WP_145853129.1
5,WP_145853129.1,WP_145853130.1
...,...,...
5243,WP_015759610.1,WP_015759614.1
5244,WP_015759610.1,WP_015759615.1
5245,WP_015759610.1,WP_015759616.1
5246,WP_015759610.1,WP_015759617.1


In [64]:
domain_for_operon = operon_prot_id.Protein_acc.value_counts()
domain_for_operon = pd.DataFrame(domain_for_operon)
domain_for_operon.columns = ['counts']
domain_for_operon

,counts
WP_132992091.1,4
WP_006553546.1,3
WP_003873519.1,3
WP_036352637.1,3
WP_067217712.1,2
...,...
WP_242612765.1,1
WP_257909987.1,1
WP_228528939.1,1
WP_194782146.1,1


In [65]:
domain_for_operon = domain_for_operon.join(pd.DataFrame(domains_names)).dropna(subset = ['domains'])
domain_for_operon['domains'] = [str(key)[1:-1] for key in domain_for_operon['domains']]
domain_for_operon = domain_for_operon.groupby('domains').sum()
domain_for_operon = domain_for_operon.sort_values(by = ['counts'], ascending=False)
domain_for_operon

,counts
domains,
'Mycofactocin_RRE',638
'mycofactocin',586
'mycofact_rSAM',581
'actino_HemFlav',354
'mycofact_creat',314
...,...
"'HTH_AsnC-type', 'MarR', 'TrmB', 'Sigma70_r4'",1
"'HTH_AsnC-type', 'MarR_2', 'HTH_20'",1
'HbpS-like',1


In [66]:
#domain_for_operon.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_arch_operon.csv')

---

__Посчитать, сколько раз во всех наших геномных регионах/оперонах одновременно присутствуют домен X и домен Y__

In [128]:
domain_counts_regions = dict()
for ind in tqdm(dom_region.index):
    buffer = dom_region.loc[ind,:]
    pair_order_list = itertools.combinations(buffer['domains'], 2)
    for pair in pair_order_list:
        if pair not in domain_counts_regions and pair[::-1] not in domain_counts_regions:
            domain_counts_regions[pair] = 1
        elif pair in domain_counts_regions:
            domain_counts_regions[pair] += 1
        elif pair[::-1] in domain_counts_regions:
            domain_counts_regions[pair[::-1]] += 1  

domain_counts_regions = pd.DataFrame({'prot_1':[key[0] for key in domain_counts_regions.keys()],'prot_2':[key[1] for key in domain_counts_regions.keys()], 'counts':domain_counts_regions.values()})
domain_counts_regions = domain_counts_regions.sort_values(by = ['counts'], ascending=False).reset_index(drop=True)
domain_counts_regions = domain_counts_regions[domain_counts_regions['counts'] >= 3]
domain_counts_regions


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 681/681 [00:00<00:00, 1507.65it/s]


,prot_1,prot_2,counts
0,Mycofactocin_RRE,mycofactocin,654
1,mycofact_rSAM,mycofactocin,646
2,mycofact_rSAM,Mycofactocin_RRE,642
3,actino_HemFlav,mycofact_rSAM,493
4,mycofactocin,mycofact_TetR,492
...,...,...,...
7272,Bac_luciferase,PP-binding,3
7273,PP-binding,menH_SHCHC,3
7274,DHPS,mycofact_TetR,3
7275,AAA_5,Pribosyltran,3


In [133]:
№domain_counts_regions.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_pair_regions.csv')

In [127]:
domain_counts_operons = dict()
for ind in tqdm(dom_operon.index):
    buffer = dom_operon.loc[ind,:]
    pair_order_list = itertools.combinations(buffer['domains'], 2)#все пары генов для данного query
    for pair in pair_order_list:
        if pair not in domain_counts_operons and pair[::-1] not in domain_counts_operons:
            domain_counts_operons[pair] = 1
        elif pair in domain_counts_operons:
            domain_counts_operons[pair] += 1
        elif pair[::-1] in domain_counts_operons:
            domain_counts_operons[pair[::-1]] += 1  #чтобы избавиться от ориентированных пар

domain_counts_operons = pd.DataFrame({'prot_1':[key[0] for key in domain_counts_operons.keys()],'prot_2':[key[1] for key in domain_counts_operons.keys()], 'counts':domain_counts_operons.values()})
domain_counts_operons = domain_counts_operons.sort_values(by = ['counts'], ascending=False).reset_index(drop=True)
domain_counts_operons = domain_counts_operons[domain_counts_operons['counts'] >= 3]
domain_counts_operons


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 675/675 [00:00<00:00, 3265.83it/s]


,prot_1,prot_2,counts
0,mycofact_rSAM,Mycofactocin_RRE,607
1,Mycofactocin_RRE,mycofactocin,576
2,mycofact_rSAM,mycofactocin,551
3,actino_HemFlav,mycofact_rSAM,354
4,actino_HemFlav,Mycofactocin_RRE,347
...,...,...,...
1351,mycofact_OYE_2,zf-C2H2_11,3
1352,aprB,GG-red-SF,3
1353,SDR_subfam_1,fumC_II,3
1354,aprB,ETF_alpha,3


In [132]:
#domain_counts_operons.to_csv('/home/niagara/Storage/Marfa/M_Zakirova/projects/mycofactocin/mftb/hmm_parsed_domain/counts/domains_pair_operons.csv')